In [14]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from json import load
import tifffile
from tqdm import tqdm
from skimage.draw import polygon
from matplotlib import rcParams
from glasbey import create_palette

palette = {
    "green": "#558150",
    "beige": "#F1E2C3",
    "brown": "#A7785A",
    "pink": "#F0D6C2",
    "black": "#0E0E0E",
}

rcParams['font.family'] = 'sans-serif'
rcParams['figure.facecolor'] = "#FFFFFF00"
rcParams['axes.facecolor'] = "#FFFFFF00"
rcParams['legend.framealpha'] = 0.2
rcParams['axes.edgecolor'] = palette["black"]
rcParams['axes.labelcolor'] = palette["black"]
rcParams['xtick.color'] = palette["black"]
rcParams['ytick.color'] = palette["black"]
rcParams['text.color'] = palette["black"]
rcParams['axes.titlecolor'] = palette["black"]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
base = Path(r"D:\Tracking\NucleiTracking\data\interim\lightsheet\2025_02_06")
# raw = tifffile.imread(list(base_path.glob("*.tif"))[1])

In [4]:
# from skimage.measure import regionprops
# from scipy.spatial.distance import cdist
# import napari
# 
# mesh = "small"
# 
# locs = tifffile.imread(base / f"{mesh}_all_locs.tif")
# masks = tifffile.imread(base / f"{mesh}_all_vals2_cp_masks.tif")
# vals = tifffile.imread(base / f"{mesh}_all_vals2.tif")
# 
# centroids = []
# 
# for t, (maskslice, locslice) in tqdm(enumerate(zip(masks, locs))): 
# 
#     frame_centroids = []
#     
#     props = regionprops(maskslice, locslice)
#     
#     for prop in props:
#         ct = [prop.centroid[0], prop.centroid[1]]
#         round_ct = [int(np.round(c)) for c in prop.centroid_local]
#         
#         ct = np.median(np.median(prop.image_intensity[round_ct[0] - 1:round_ct[0] + 2, round_ct[1] - 1:round_ct[1] + 2], axis=0), axis=0)
#         # ct = prop.image_intensity[round_ct[0], round_ct[1]]
#         
#         frame_centroids.append([t, *ct])
#                 
#     
#     centroids.extend(frame_centroids)
#                 
# centroids = np.array(centroids)
# 
# viewer = napari.Viewer()
# pts = viewer.add_points(centroids, shading="spherical")
# 
# napari.run()

242it [02:09,  1.87it/s]


In [17]:
from skimage.measure import regionprops_table
from scipy.spatial.distance import cdist
import napari

mesh = "small"

locs = tifffile.imread(base / f"{mesh}_all_locs.tif")
masks = tifffile.imread(base / f"{mesh}_all_vals2_cp_masks.tif")
vals = tifffile.imread(base / f"{mesh}_all_vals2.tif")

centroids = []

for t, (maskslice, locslice) in tqdm(enumerate(zip(masks, locs))): 

    # frame_centroids = []
    
    cts = regionprops_table(maskslice, locslice, properties=("centroid","intensity_mean"))
    # print(cts.keys())
    
    locs = np.round(np.stack([[t]*len(cts["intensity_mean-0"]), 
                              cts["intensity_mean-0"], 
                              cts["intensity_mean-1"], 
                              cts["intensity_mean-2"]], axis=-1)).astype(np.uint16)
    
    
        # ct = [prop.centroid[0], prop.centroid[1]]
        # round_ct = [int(np.round(c)) for c in prop.centroid_local]
        # 
        # ct = np.median(np.median(prop.image_intensity[round_ct[0] - 1:round_ct[0] + 2, round_ct[1] - 1:round_ct[1] + 2], axis=0), axis=0)
        # # ct = prop.image_intensity[round_ct[0], round_ct[1]]
        # 
        # frame_centroids.append([t, *ct])
    
    centroids.extend([l for l in locs])
                
    
    # centroids.extend(frame_centroids)
                
# centroids = np.array(centroids)

viewer = napari.Viewer()
pts = viewer.add_points(centroids, shading="spherical")

napari.run()

154it [00:12,  6.81it/s]C:\Users\Harrison\AppData\Local\Temp\ipykernel_4036\2657269221.py:23: RuntimeWarning: invalid value encountered in cast
  cts["intensity_mean-2"]], axis=-1)).astype(np.uint16)
269it [00:48,  5.56it/s]


In [19]:
import pandas as pd
centroids = np.array(centroids)

cdf = {"frame": centroids[:, 0], "z": centroids[:, 1], "y": centroids[:, 2], "x": centroids[:, 3]}
df = pd.DataFrame(cdf)
df["frame"] = np.array(centroids[:, 0], dtype=int)
df.to_csv(base / f"{mesh}_spots.csv")